### Scaling - Cedric Liang

#### Image Selection

Now that our science images have been processed according to the procedure described in the opening chapters of Rieke, we must process the images further to remove problematic systemic issues in our data.

A great deal of our images were taken in imperfect conditions without the usage of the auto-tracker. As such, there is noticeable movement of the open cluster in our images, and we must computationally correct for these imperfections.

However, it first makes sense to get a statistical overview of the images and use some common sense to discard the ones that are clearly unfit for use. There was intermittent cloud cover on the night of our observations, and as such a reasonable portion (30-40%) of the images were taken such that the background was significantly elevated due to the reflection of terrestrial light - in some cases up to 20,000 counts.

As such, it makes sense for us to first analyse our data and determine which images to simply discard.

There is also a human element to this - it is more difficult for computational methods to discard qualitatively poor images that can impact our results, such as images with poor focus or streaking. Indeed, we noticed on the night of observation that some images had trails. With the quantity of data that we have, it makes more sense to discard these manually.


In [42]:
import os
import warnings
import numpy as np
import ccdproc
from ccdproc import CCDData
from photutils.centroids import centroid_2dg
from scipy.ndimage import shift
import gc
gc.enable()

warnings.filterwarnings('ignore')
ROOT_PATH = os.path.normpath(os.getcwd() + os.sep + os.pardir)

# key values for our bands
BANDS = ["V", "B", "R"]

# redefining functions since importing across notebooks seems to be unreliable


def image_stats(image):
    return {
        'Min': np.min(image),
        'Max': np.max(image),
        'Mean': np.mean(image),
        'Mdn': np.median(image),
        'Stdev': np.std(image)
    }


def print_stats(label: str, stats_dict):
    print("\n", label)
    for key, value in stats_dict.items():
        print("\t", key, "\t\t", value)


**[Nico]** I started with the V-band and viewed the images in ds9 to visually inspect these images for tracking and focussing errors.

V-band images:
Looks good: 3862-3871, 3922, 3923, 3925, 3926, 3927, 4052, 4062, 4064, 4098, 4099\
Slight tracking errors: 3929, 4053, 4063\
Big tracking errors: 3920, 3921, 3924, 3928, 4065\
Lots of cloud: 4058, 4059

Then I looked at the B-band:

Looks good: 3852-3861, 3882, 3885, 3886, 3889, 3890, 3942, 3943, 3946, 3947, 3950, 3951\
Slight focusing errors: 3816-3825\
Slight tracking errors: 3816, 3883, 3887, 3891, 3944, 3948\
Big tracking errors: 3884, 3888, 3945, 3949

**[Cedric from here onwards]** Selecting R-band images - we filter based on qualitative suitability, with a preference for filtering out the images with severe streaking. We can filter the cloud cover images statistically/computationally later on, but a lack of focus/streaking is harder to do.

- 😁 Great : 3872-3881, 4056, 
- 🤨 Okay: 4094, 4095, 3937, 3933, 4057, 
- 😞 Use if must: 3930, 4060, 



Image 4056 here has a relatively elevated background count, but it seems to be the sharpest/the one most in focus and with the least tracking issues. As such, it seems to be the most suitable for shifting as the centroid calculation would be the most reliable.

Ok, we have a reference star. Let's go through the process of shifting based on this star.

In [43]:
# Image numbers that are acceptable for each band

good_images = {
    "B": list(range(3852, 3862)) + [3885, 3886, 3889, 3890, 3942, 3943, 3946, 3947, 3950, 3951],
    "V": list(range(3862, 3872)) + [3922, 3923] + list(range(3925, 3928)) + [4052, 4062, 4064, 4098, 4099],
    "R": list(range(3872, 3882)) + [3933, 3937, 4056, 4057, 4094, 4095]
}


In [44]:
proc_files = {band: ccdproc.ImageFileCollection(
    f"{ROOT_PATH}/src/processed_ims/", glob_include=f"proc_NGC_3293_{band}*") for band in BANDS}

# idx: 20:24 represents the four digits in the file names that represent the file number
# here we're reading only the files that we have selected
scim = {
    band: [CCDData.read(f"{ROOT_PATH}/src/processed_ims/{fn}")
           for fn in image_files.files_filtered(PICTTYPE=1)
           if int(fn[20:24]) in good_images[band]]
    for band, image_files
    in proc_files.items()}

# number of images we have for each band
print(len(scim["V"]), len(scim["B"]), len(scim["R"]))
# print(new_names)


20 20 16


We'll also copy these selected images to a new directory so that if we need to access them with ds9, we don't have to go sorting.

In [45]:
import contextlib
with contextlib.suppress(OSError):
    os.mkdir(f"{ROOT_PATH}/src/processed_ims/selected")


In [46]:
import shutil

temp_names = {band: [fn for fn in image_files.files_filtered(PICTTYPE=1) if int(
    fn[20:24]) in good_images[band]] for band, image_files in proc_files.items()}

for band, files in temp_names.items():
    for filename in files:
        shutil.copyfile(f"{ROOT_PATH}/src/processed_ims/{filename}",
                        f"{ROOT_PATH}/src/processed_ims/selected/{filename}")


Let's now shift these images. Unfortunately, we have a huge amount of tracking error/object relative displacement throughout the night, and after viewing the images in ds9, it becomes apparent that it's impossible to define a single shift for these images since the extremes of the locations of some of the objects across the objects exceeds the size of any region in which there is a single star and not much else.

As such, we will have to perform shifting in two stages - the first is to group 'similarly positioned' objects and shift them within the group (relative to each other), then the next step will be to perform a global shift using the displacements for each group to produce a globally shifted image. 

Well - we'll see what we get after the first stage shifts - it could be the case that we need a few iterative stages of shifting.

I manually went through ds9 and grouped the stars into the following groups, along with the boxes defined that contain a single star within them for each group, and as such can be used for an offset.


**B band**
- Group 0: [3852-3861]: (437-485, 1171-1204)
- Group 1: [3885-3886, 3889-3890]: (1194-1258, 1081-1141)
- Group 2: [3942-3943, 3946-3947]: (1296-1350, 1001-1066)
- Group 3: [3950-3951]: (440-465, 1197-1232)

**R band**
- Group 4: [3872-3881]: (1266-1309, 1046-1082)
- Group 5: [3933, 3937]: (746-789, 1102-1151)
- Group 6: [4056-4057, 4094-4095]: (1335-1373, 987-1035) - Note that 4056 is the intended global reference star, so ensure that all objects in this group are also shifted relative to 4056

**V band**
- Group 7: [3862-3871]: (1613-1657, 403-454)
- Group 8: [3922-3923, 3925-3926, 4098-4099]: (1292-1354, 998-1059)
- Group 9: [4052]
- Group 10 [4062, 4064]: (19-100, 791-874)

In [47]:
# define groups
# index is group number
# zeroeth index is list of image numbers
# first index is tuple of coords
groups = [
    (list(range(3852, 3862)), (437, 485), (1171, 1204)),
    ([3885, 3886, 3889, 3890], (1194, 1258), (1081, 1141)),
    ([3942, 3943, 3946, 3947], (1299, 1348), (1001, 1062)),
    ([3950, 3951], (440, 465), (1197, 1232)),

    (list(range(3872, 3882)), (1266, 1309), (1046, 1082)),
    ([3933, 3937], (670, 720), (35, 88)),
    ([4056, 4057, 4094, 4095], (1335, 1373), (987, 1035)),

    (list(range(3862, 3872)), (1613, 1657), (403, 454)),
    ([3922, 3923, 3925, 3926, 4098, 4099], (1292, 1354), (998, 1059)),
    ([4052], (100, 200), (100, 200)),
    ([4062, 4064], (952, 981), (279, 322))
]

local_image_files = ccdproc.ImageFileCollection(
    f"{ROOT_PATH}/src/processed_ims/", glob_include=f"proc_NGC_3293_*")

scim_groups = []
for group in groups:
    group_image_numbers = group[0]

    group_scims = [CCDData.read(f"{ROOT_PATH}/src/processed_ims/{fn}")
                   for fn in local_image_files.files_filtered(PICTTYPE=1)
                   if int(fn[20:24]) in group_image_numbers]

    scim_groups.append(group_scims)


In [48]:
shifts = []

for idx in range(len(groups)):
    group_scim = scim_groups[idx]
    x_range = groups[idx][1]
    y_range = groups[idx][2]

    xoffset = x_range[0]
    yoffset = y_range[0]
    xbox = x_range[1] - xoffset
    ybox = y_range[1] - yoffset

    shiftx = []
    shifty = []

    for local_image in group_scim:
        temp = local_image.copy()
        temp = temp-np.ma.median(temp)
        x1, y1 = centroid_2dg(
            temp[yoffset: yoffset + ybox, xoffset: xoffset + xbox])

        shiftx.append(x1 + xoffset)
        shifty.append(y1 + yoffset)

    # shift relative to the first image in each group - to ensure that 4056 is used as the reference for its group.
    shifts_zipped = list(map(lambda tup: (round(tup[0]), round(
        tup[1])), zip(shiftx[0]-shiftx, shifty[0]-shifty)))
    shifts.append(shifts_zipped)

for idx, sft in enumerate(shifts):
    print(f"Group {idx}: {sft}")


Group 0: [(0, 0), (0, 0), (2, 0), (0, 0), (3, -1), (7, -2), (8, -2), (11, -3), (11, -3), (15, -5)]
Group 1: [(0, 0), (2, -6), (29, -24), (26, -28)]
Group 2: [(0, 0), (2, -7), (21, -28), (22, -35)]
Group 3: [(0, 0), (3, -9)]
Group 4: [(0, 0), (2, -2), (4, -2), (5, -3), (7, -5), (8, -6), (11, -7), (12, -8), (14, -9), (17, -8)]
Group 5: [(0, 0), (24, -26)]
Group 6: [(0, 0), (2, -7), (2, -3), (5, -10)]
Group 7: [(0, 0), (0, 0), (1, -1), (0, -2), (0, -3), (0, -3), (0, -4), (1, -5), (1, -5), (1, -6)]
Group 8: [(0, 0), (-1, -3), (21, -18), (29, -26), (2, -33), (6, -34)]
Group 9: [(0, 0)]
Group 10: [(0, 0), (4, -15)]


Let's now perform our shifts.

In [49]:
new_file_names = {int(fn[20:24]): fn
                  for fn in local_image_files.files
                  }

# uncomment this if you need to see intermediary files - but I've commented it to save space
# with contextlib.suppress(OSError):
#     os.mkdir(f"{ROOT_PATH}/src/shift_stage1/")


In [50]:
first_stage_shifted = []

for idx in range(len(groups)):
    group_scim = scim_groups[idx]
    group_shifted = []
    for image_idx, local_image in enumerate(group_scim):
        # Note the y-x convention being used here and in the following command.
        yxshifts = (shifts[idx][image_idx][1], shifts[idx][image_idx][0])

        temp = CCDData(shift(local_image, yxshifts, order=0, mode='constant',
                       cval=-1000)-np.ma.median(local_image), unit="adu")
        temp.header = local_image.header
        group_shifted.append(temp)
        new_file_name = "s1" + new_file_names[groups[idx][0][image_idx]]
        new_file_dir = f"{ROOT_PATH}/src/shift_stage1/"

        # uncomment this if you need to see intermediary files - but I've commented it to save space
        # temp.write(new_file_dir + new_file_name, overwrite=True)
    first_stage_shifted.append(group_shifted)


We have performed our first stage shift - let's take a look at them in ds9. We should expect to see images in the same group have little translation relative to each other.

We find that images within a group are shifted - not perfectly, since the size of the boxes we defined and the magnitude of the translation we needed to account for potentially introduced some issues with regards to centroid calculation - perhaps the 'corona' of another object is within our boxes, causing the shifts to be a pixel or two off sometimes.

That's not an issue though - we'll perform a final shift later on once we've aligned the groups in order to get rid of this small scale error. We'll now correct for the large scale error manually.

We'll select a distinct star in our reference image 4056 - bright enough to be distinguised in all all our images, but faint enough so that it doesn't illuminate too many pixels, which would reduce the 'precision' of our manual eyeball shift. I'll select the star at 681, 409 in that image.

Now, we'll go through each group and manually list the approximate coordinates of that star in that group. That will define the offset from 681, 409 that will be applied to the entire group.

Group 0: (631, 448)\
Group 1: (564, 499)\
Group 2: (658, 417)\
Group 3: (618, 470)

Group 4: (615, 464)\
Group 5: (590, 490)\
Group 6: (681, 409) - REFERENCE

Group 7: (612, 456)\
Group 8: (658, 416)\
Group 9: (575, 524)\
Group 10: (634, 466)

In [51]:
# offset for each
location_ref = (681, 409)

actual_loc = [
    (631, 448),
    (564, 499),
    (658, 417),
    (618, 470),
    (615, 464),
    (590, 490),
    (681, 409),
    (612, 456),
    (658, 416),
    (575, 524),
    (634, 466)
]

group_offsets = [
    (-1*(tup[0]-location_ref[0]), -1*(tup[1]-location_ref[1]))
    for tup in actual_loc
]

print(group_offsets)


[(50, -39), (117, -90), (23, -8), (63, -61), (66, -55), (91, -81), (0, 0), (69, -47), (23, -7), (106, -115), (47, -57)]


Now that we have the offset for each group, we can perform the shift.

In [52]:
# uncomment this if you need to see intermediary files - but I've commented it to save space
# with contextlib.suppress(OSError):
#     os.mkdir(f"{ROOT_PATH}/src/shift_stage2/")


In [53]:
second_stage_shifted = []

for idx in range(len(groups)):
    group_scim = first_stage_shifted[idx]
    group_shifted = []
    for image_idx, local_image in enumerate(group_scim):
        # Note the y-x convention being used here and in the following command.
        yxshifts = (group_offsets[idx][1], group_offsets[idx][0])

        temp = CCDData(shift(local_image, yxshifts, order=0,
                       mode='constant', cval=-1000), unit="adu")
        temp.header = local_image.header
        group_shifted.append(temp)
        new_file_name = "s2" + new_file_names[groups[idx][0][image_idx]]
        new_file_dir = f"{ROOT_PATH}/src/shift_stage2/"

        # uncomment this if you need to see intermediary files - but I've commented it to save space
        # temp.write(new_file_dir + new_file_name, overwrite=True)

    second_stage_shifted.append(group_shifted)


Inspection with DS9 shows that the large scale errors are no longer there. We'll shift one more time to get rid of the small scale issues we noticed before - it's much easier now since we can flatten the group and choose a single star that's present in all the images, as well as use a relatively small box.

We'll use the box
    (902-942, 488-527)

In [54]:
xoffset = 902
yoffset = 488
xbox = 942 - xoffset
ybox = 527 - yoffset

reference_x = None
reference_y = None
shifts_final = []

for idx in range(len(groups)):
    group_scim = second_stage_shifted[idx]

    shiftx = []
    shifty = []

    for local_image in group_scim:
        temp = local_image.copy()
        temp = temp-np.ma.median(temp)
        x1, y1 = centroid_2dg(
            temp[yoffset: yoffset + ybox, xoffset: xoffset + xbox])

        shiftx.append(x1 + xoffset)
        shifty.append(y1 + yoffset)

    # now we only want to refer to the first image out of all of them, not the first image in each group
    reference_x = shiftx[0] if not reference_x else reference_x
    reference_y = shifty[0] if not reference_y else reference_y

    shifts_zipped = list(map(lambda tup: (round(tup[0]), round(
        tup[1])), zip(reference_x-shiftx, reference_y-shifty)))
    shifts_final.append(shifts_zipped)

for idx, sft in enumerate(shifts_final):
    print(f"Group {idx}: {sft}")


Group 0: [(0, 0), (1, 0), (0, -1), (3, -2), (2, -1), (0, 0), (0, 0), (-1, 0), (0, 0), (-3, 1)]
Group 1: [(0, 0), (1, 0), (-4, 2), (1, 0)]
Group 2: [(0, -1), (-1, -1), (-1, -1), (-1, -1)]
Group 3: [(-1, -2), (-1, -2)]
Group 4: [(1, -4), (0, -2), (-1, -3), (-1, -2), (-2, -1), (-2, 0), (-4, 0), (-4, 1), (-5, 1), (-6, 0)]
Group 5: [(-1, 0), (-1, -1)]
Group 6: [(0, -2), (-1, 0), (-2, -1), (-2, -1)]
Group 7: [(1, -1), (2, -2), (1, -1), (2, -1), (2, -1), (2, -2), (2, -1), (2, -1), (2, -2), (2, -2)]
Group 8: [(-1, -1), (3, -4), (-1, -1), (-5, 2), (2, 6), (-1, 0)]
Group 9: [(-1, -2)]
Group 10: [(-1, -2), (-1, -2)]


In [55]:
with contextlib.suppress(OSError):
    os.mkdir(f"{ROOT_PATH}/src/shift_final/")

for idx in range(len(groups)):
    group_scim = second_stage_shifted[idx]
    for image_idx, local_image in enumerate(group_scim):
        # Note the y-x convention being used here and in the following command.
        yxshifts = (shifts_final[idx][image_idx][1],
                    shifts_final[idx][image_idx][0])

        temp = CCDData(shift(local_image, yxshifts, order=0,
                       mode='constant', cval=-1000), unit="adu")
        temp.header = local_image.header
        new_file_name = "s" + new_file_names[groups[idx][0][image_idx]]
        new_file_dir = f"{ROOT_PATH}/src/shift_final/"

        temp.write(new_file_dir + new_file_name, overwrite=True)


Let's check our shifts in ds9 - they look good! Little to no translation in ds9, which means our shifts have been successful and our images are ready to be combined.